# Neptune + Skorch

## Introduction

This guide will show you how to:

* Create a `NeptuneLogger()`,
* Log training metrics to Neptune using `NeptuneLogger()`.

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects)

## Install Neptune and dependencies

In [ ]:
! pip install neptune-client>=0.11.0 neptune-skorch torch==1.13.0 scikit-learn==1.1.3 skorch==0.12.1 

## Import libraries

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import neptune.new as neptune
from neptune.new.types import File
from skorch.callbacks import NeptuneLogger, Checkpoint
from skorch import NeuralNetClassifier

## Loading Data
Using SciKit-Learns ```fetch_openml``` to load MNIST data.

In [ ]:
mnist = fetch_openml('mnist_784', as_frame=False, cache=False)

## Preprocessing Data

Each image of the MNIST dataset is encoded in a 784 dimensional vector, representing a 28 x 28 pixel image. Each pixel has a value between 0 and 255, corresponding to the grey-value of a pixel.<br />
The above ```featch_mldata``` method to load MNIST returns ```data``` and ```target``` as ```uint8``` which we convert to ```float32``` and ```int64``` respectively.

In [ ]:
X = mnist.data.astype('float32')
y = mnist.target.astype('int64')

To avoid big weights that deal with the pixel values from between [0, 255], we scale `X` down. A commonly used range is [0, 1].

In [ ]:
X /= 255.0

In [ ]:
X.min(), X.max()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
assert(X_train.shape[0] + X_test.shape[0] == mnist.data.shape[0])

In [ ]:
X_train.shape, y_train.shape

### Print a selection of training images and their labels

In [ ]:
def plot_example(X, y):
    """Plot the first 5 images and their labels in a row."""
    for i, (img, y) in enumerate(zip(X[:5].reshape(5, 28, 28), y[:5])):
        plt.subplot(151 + i)
        plt.imshow(img)
        plt.xticks([])
        plt.yticks([])
        plt.title(y)

In [ ]:
plot_example(X_train, y_train)

## Build Neural Network with PyTorch
Simple, fully connected neural network with one hidden layer. Input layer has 784 dimensions (28x28), hidden layer has 98 (= 784 / 8) and output layer 10 neurons, representing digits 0 - 9.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
mnist_dim = X.shape[1]
hidden_dim = int(mnist_dim/8)
output_dim = len(np.unique(mnist.target))

In [ ]:
mnist_dim, hidden_dim, output_dim

A Neural network in PyTorch's framework.

In [ ]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            input_dim=mnist_dim,
            hidden_dim=hidden_dim,
            output_dim=output_dim,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.dropout = nn.Dropout(dropout)

        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, X, **kwargs):
        X = F.relu(self.hidden(X))
        X = self.dropout(X)
        X = F.softmax(self.output(X), dim=-1)
        return X

## Initialize Neptune run

Connect your script to Neptune application and create new run.

In [ ]:
run = neptune.init_run(
    api_token=neptune.ANONYMOUS_API_TOKEN,
    project='common/skorch-integration',
    name='skorch-example'
)

You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune `project`.

At this point you have new run in Neptune. For now on you will use `run` to log metadata to it.

---

**Note**


Instead of logging data to the public project `'common/skorch-integration'` as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your [Neptune API token](https://docs.neptune.ai/getting-started/installation#get-api-token)
2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your project to the ``project`` argument of ``neptune.init()``.

For example:

```python
neptune.init(project="YOUR_WORKSPACE/YOUR_PROJECT",
             api_token="YOUR_API_TOKEN")
```

## Create NeptuneLogger 

In [ ]:

neptune_logger = NeptuneLogger(run, close_after_train=False)

In [ ]:
checkpoint_dirname = './checkpoints'
checkpoint = Checkpoint(dirname=checkpoint_dirname)

## Initialize a trainer and pass neptune_logger

In [ ]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    device=device,
    callbacks=[neptune_logger, checkpoint]
)

In [ ]:
net.fit(X_train, y_train);

## More Options

### Log model weights
You need to have saved these files to disk using the Checkpoint Callback before trying to upload to Neptune 

In [ ]:
neptune_logger.run["training/checkpoints"].upload_files(checkpoint_dirname)

### Log prediction score

In [ ]:
y_pred = net.predict(X_test)
neptune_logger.run["training/acc"] = accuracy_score(y_test, y_pred)

### Log misclassified images

In [ ]:
error_mask = y_pred != y_test
plot_example(X_test[error_mask], y_pred[error_mask])

for (x,y_hat, y) in zip(X_test[error_mask], y_pred[error_mask], y_test[error_mask]):
    x_reshaped = x.reshape(28, 28)
    neptune_logger.run["training/test/misclassified_images"].log(File.as_image(x_reshaped), description=f"y_pred={y_hat}, y_true={y}")

## Stop logging
Once you are done logging, stop tracking the run.

In [ ]:
neptune_logger.run.stop()

## Analyze logged metadata in the Neptune app

Follow the link to the run and explore metadata (metrics, hyperparams, model checkpoints) that were logged to the run in Neptune.

The link should look like this: https://app.neptune.ai/o/common/org/skorch-integration/e/SKOR-13